In [1]:
import pandas as pd

In [2]:
df =pd.read_csv(r"preprocessed.csv")
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0,0,0,5849,0.0,128.0,360.0,1.0,2,1
1,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,0
2,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,1
3,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,1
4,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,1


<h2 style="color:green" >  Outliers detection and removal using Percentile method :</h2>

<h3 style="color:green" >  Lets apply quantile range on price column</h3>

In [3]:
min_th_pr,max_th_pr=df['LoanAmount'].quantile([0.01,0.99])
min_th_pr,max_th_pr

(30.65, 495.87)

In [4]:
print("lower values outliers :",len(df[df['LoanAmount']<min_th_pr]))
print("upper values outlierst :",len(df[df['LoanAmount']>max_th_pr]))

lower values outliers : 7
upper values outlierst : 7


In [5]:
outliers = df[(df.LoanAmount>max_th_pr) | (df.LoanAmount < min_th_pr)]
outliers.shape

(14, 12)

In [6]:
outliers['LoanAmount']

14      17.0
94      25.0
130    650.0
133     25.0
147     30.0
155    600.0
171    700.0
270     30.0
369    570.0
487    500.0
555     26.0
561    600.0
568      9.0
604    496.0
Name: LoanAmount, dtype: float64

In [7]:
df_new = df[(df['LoanAmount']<max_th_pr) & (df['LoanAmount']>min_th_pr)]
print("with outliers shape is :",df.shape)
print("without outliers shape is",df_new.shape)

with outliers shape is : (614, 12)
without outliers shape is (600, 12)


<h1 style="color:green" >  Outliers detection and removal using Z-score method :</h1><br>
<h4 style="color:green">upper_lmt : means any point greator than upper_lmt will be consider as outlier .</h4>
<h4 style="color:green">lower_lmt : means any point below  lower_lmt will be consider as outlier .

In [8]:
#I am taking 3 std dev
upper_lmt = df.ApplicantIncome.mean()+3*df.ApplicantIncome.std()
lower_lmt = df.ApplicantIncome.mean()-3*df.ApplicantIncome.std()
print(upper_lmt,lower_lmt)

23730.584303549156 -12923.66573677391


In [9]:
outliers = df[(df.ApplicantIncome>upper_lmt) | (df.ApplicantIncome < lower_lmt)]
outliers.shape

(8, 12)

In [10]:
df_new_z = df[(df['ApplicantIncome']<upper_lmt) & (df['ApplicantIncome']>lower_lmt)]
print("with outliers shape is :",df.shape)
print("without outliers shape is",df_new_z.shape)

with outliers shape is : (614, 12)
without outliers shape is (606, 12)


<h1 style="color:green" >  Outliers detection and removal using IQR method :</h1><br>
<h4 style="color:green">upper_lmt : means any point greator than upper_lmt will be consider as outlier .</h4>
<h4 style="color:green">lower_lmt : means any point below  lower_lmt will be consider as outlier .

##  IQR
* Q1 = 25th Persentile
* Q2 = 75th Persentile
* 
#### IQR = Q3 - Q1
#### lower_limit  = Q1 - 1.5*IQR
#### upper_limit  = Q3 + 1.5*IQR

In [11]:
Q1 = df.CoapplicantIncome.quantile(0.25)
Q3 = df.CoapplicantIncome.quantile(0.75)
print(Q1,Q3)

0.0 2297.25


In [12]:
IQR = Q3 - Q1
IQR

2297.25

In [13]:
lower_limit  = Q1 - 1.5*IQR
upper_limit  = Q3 + 1.5*IQR
print(lower_limit,upper_limit)

-3445.875 5743.125


In [14]:
outliers = df[(df.CoapplicantIncome>upper_limit) | (df.CoapplicantIncome < lower_limit)]
outliers.shape

(18, 12)

In [15]:
df_new = df[(df['CoapplicantIncome']<upper_limit) & (df['CoapplicantIncome']>lower_limit)]
print("with outliers shape is :",df.shape)
print("without outliers shape is",df_new.shape)

with outliers shape is : (614, 12)
without outliers shape is (596, 12)


## I am going to use IQR for finally outliers detection and removal 

In [16]:
Q1 = df[['ApplicantIncome','CoapplicantIncome','LoanAmount']].quantile(0.25)
Q3 = df[['ApplicantIncome','CoapplicantIncome','LoanAmount']].quantile(0.75)
print("First quartile (Q1) :\n" ,Q1)
print()
print("Third quartile (Q3) :\n" ,Q3)

First quartile (Q1) :
 ApplicantIncome      2877.50
CoapplicantIncome       0.00
LoanAmount            100.25
Name: 0.25, dtype: float64

Third quartile (Q3) :
 ApplicantIncome      5795.00
CoapplicantIncome    2297.25
LoanAmount            164.75
Name: 0.75, dtype: float64


In [17]:
df['LoanAmount'].quantile(0.75)

164.75

In [18]:
IQR = Q3 - Q1
IQR

ApplicantIncome      2917.50
CoapplicantIncome    2297.25
LoanAmount             64.50
dtype: float64

In [19]:
lower_limit  = Q1 - 1.5*IQR
upper_limit  = Q3 + 1.5*IQR
print(lower_limit)
print(upper_limit)

ApplicantIncome     -1498.750
CoapplicantIncome   -3445.875
LoanAmount              3.500
dtype: float64
ApplicantIncome      10171.250
CoapplicantIncome     5743.125
LoanAmount             261.500
dtype: float64


In [20]:
# Unpacking 
ApplicantIncome_upper,CoapplicantIncome_upper,LoanAmount_upper = upper_limit
ApplicantIncome_lower,CoapplicantIncome_lower,LoanAmount_lower = lower_limit

In [21]:
outliers_ApplicantIncome = df[(df.ApplicantIncome>ApplicantIncome_upper) | (df.ApplicantIncome < ApplicantIncome_lower)]
outliers_CoapplicantIncome = df[(df.CoapplicantIncome>CoapplicantIncome_upper) | (df.CoapplicantIncome < CoapplicantIncome_lower)]
outliers_LoanAmount = df[(df.LoanAmount>LoanAmount_upper) | (df.LoanAmount < LoanAmount_lower)]

In [22]:
print(outliers_ApplicantIncome.shape)
print(outliers_CoapplicantIncome.shape)
print(outliers_LoanAmount.shape)

(50, 12)
(18, 12)
(41, 12)


<h3 style="color:green" > Lets  remove outliers (wrt ApplicantIncome ) ↓

In [24]:
df_new = df[(df['ApplicantIncome']<ApplicantIncome_upper) & (df['ApplicantIncome']>ApplicantIncome_lower)]

print("with outliers shape is :",df.shape)
print("without outliers shape is",df_new.shape)

with outliers shape is : (614, 12)
without outliers shape is (564, 12)


<h3 style="color:green" > Lets  remove outliers (wrt CoapplicantIncome ) ↓

In [25]:
df_new = df[(df['CoapplicantIncome']<CoapplicantIncome_upper) & (df['ApplicantIncome']>CoapplicantIncome_lower)]

print("with outliers shape is :",df.shape)
print("without outliers shape is",df_new.shape)

with outliers shape is : (614, 12)
without outliers shape is (596, 12)


<h3 style="color:green" > Lets  remove outliers (wrt LoanAmount ) ↓

In [26]:
df_new = df[(df['LoanAmount']<LoanAmount_upper) & (df['LoanAmount']>LoanAmount_lower)]

print("with outliers shape is :",df.shape)
print("without outliers shape is",df_new.shape)

with outliers shape is : (614, 12)
without outliers shape is (573, 12)


<h3 style="color:green" > Dataset after removing outliers : ↓

In [27]:
final_df = df[(df['ApplicantIncome']<ApplicantIncome_upper) & (df['ApplicantIncome']>ApplicantIncome_lower)]
final_df = final_df[(final_df['CoapplicantIncome']<CoapplicantIncome_upper) & (final_df['ApplicantIncome']>CoapplicantIncome_lower)]
final_df = final_df[(final_df['LoanAmount']<LoanAmount_upper) & (final_df['LoanAmount']>LoanAmount_lower)]

In [28]:
final_df.shape

(535, 12)

In [29]:
final_df.corr()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Gender,1.000000,0.375571,0.187076,0.072320,0.021148,0.074849,0.198652,0.147324,-0.079050,0.005714,-0.006906,0.047217
Married,0.375571,1.000000,0.347329,0.032004,0.020443,-0.007531,0.266574,0.197150,-0.090463,0.003310,0.007718,0.077326
Dependents,0.187076,0.347329,1.000000,0.082991,0.054542,0.118185,-0.064006,0.113234,-0.092739,-0.032058,0.016343,0.008150
Education,0.072320,0.032004,0.082991,1.000000,0.000869,-0.147857,-0.028559,-0.122120,-0.073561,-0.085002,-0.063382,-0.114159
Self_Employed,0.021148,0.020443,0.054542,0.000869,1.000000,0.187889,-0.056586,0.086229,-0.064348,0.031554,-0.060971,-0.005302
ApplicantIncome,0.074849,-0.007531,0.118185,-0.147857,0.187889,1.000000,-0.266763,0.476023,-0.062157,0.051843,-0.076389,0.013046
CoapplicantIncome,0.198652,0.266574,-0.064006,-0.028559,-0.056586,-0.266763,1.000000,0.290349,-0.008596,0.008528,-0.092128,0.068056
LoanAmount,0.147324,0.197150,0.113234,-0.122120,0.086229,0.476023,0.290349,1.000000,0.089344,0.010476,-0.120310,-0.013972
Loan_Amount_Term,-0.079050,-0.090463,-0.092739,-0.073561,-0.064348,-0.062157,-0.008596,0.089344,1.000000,-0.017483,-0.081467,-0.025424
Credit_History,0.005714,0.003310,-0.032058,-0.085002,0.031554,0.051843,0.008528,0.010476,-0.017483,1.000000,0.009835,0.566580


In [30]:
df.corr()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Gender,1.000000,0.367389,0.172914,0.045364,-0.000525,0.058809,0.082912,0.106904,-0.074030,0.009170,-0.025752,0.017987
Married,0.367389,1.000000,0.340684,0.017671,0.008491,0.050194,0.080496,0.145717,-0.103381,0.006690,0.000395,0.084281
Dependents,0.172914,0.340684,1.000000,0.055752,0.056798,0.118202,0.030430,0.163103,-0.103864,-0.040160,-0.000244,0.010118
Education,0.045364,0.017671,0.055752,1.000000,-0.010383,-0.140760,-0.062290,-0.168759,-0.073928,-0.073658,-0.065243,-0.085884
Self_Employed,-0.000525,0.008491,0.056798,-0.010383,1.000000,0.127180,-0.016100,0.115100,-0.033739,-0.001550,-0.030860,-0.003700
ApplicantIncome,0.058809,0.050194,0.118202,-0.140760,0.127180,1.000000,-0.116605,0.565181,-0.046531,-0.018615,-0.009500,-0.004710
CoapplicantIncome,0.082912,0.080496,0.030430,-0.062290,-0.016100,-0.116605,1.000000,0.189218,-0.059383,0.011134,0.010522,-0.059187
LoanAmount,0.106904,0.145717,0.163103,-0.168759,0.115100,0.565181,0.189218,1.000000,0.036960,-0.000607,-0.046632,-0.033214
Loan_Amount_Term,-0.074030,-0.103381,-0.103864,-0.073928,-0.033739,-0.046531,-0.059383,0.036960,1.000000,-0.004705,-0.076120,-0.022549
Credit_History,0.009170,0.006690,-0.040160,-0.073658,-0.001550,-0.018615,0.011134,-0.000607,-0.004705,1.000000,0.001963,0.540556


In [31]:
final_df.to_csv('train.csv',index=False)

In [32]:
dff = pd.read_csv('train.csv')
dff.shape

(535, 12)